In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages
# use curve_fit for Rosenbluth linear fit
from scipy.optimize import curve_fit
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# optional: use science paper style for plots
# from matplotlib.ticker import ScalarFormatter

import scienceplots
plt.style.use('science')

import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [2]:
# notations in this code:
# nu: energy transfer
# cross: cross section
# Q2: four-momentum transfer squared
# qv: bold q, three-momentum transfer
# Ex: excitation energy
# W: invariant mass of the final state (W2: W squared)
# epsilon: virtual photon polarization parameter
# Theta: scattering angle
# E0: incident electron energy
# Ep: final electron energy
# Veff: effective potential (_eff is used for "effective" quantities)

In [3]:
# preset values for the analysis: (you need to update)
#____________________________________________________________________________________________________________________
# physical constants
A = 56
Z = 26
C12_factor = Z / 6
mass_nucleon = 0.938273
mass_nucleus = A * 0.931494
alpha_fine = 1 / 137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']


In [4]:
# utility functions:

# round to only 3 significant figures (for printing)
def round_sig_3(x):
    return '%s' % float('%.3g' % x)

# append a row to a dataframe
def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

# linear model for Rosenbluth fitting
def linear_model(x, a, b):
    return a * x + b

# prepare the dataframe
# "filepath" should be the path to the csv file that contains columns {'A', 'Z', 'nu', 'E0', 'ThetaDeg', 'cross', 'error', 'dataSet'}
# read dataframe from csv file
def prepare_df(filepath):

    df = pd.read_csv(filepath)
    initial_row_count = len(df)
    df = df.dropna(subset=['bc_qv_ex', 'bc_qv_w2'])
    df = df[~np.isinf(df['bc_qv_ex']) & ~np.isinf(df['bc_qv_w2'])]
    df.reset_index(drop=True, inplace=True)
    final_row_count = len(df)
    rows_dropped = initial_row_count - final_row_count
    print(f"Number of rows dropped: {rows_dropped}")

    return df

In [5]:
# load from a pre-computed data base
df = prepare_df('Data/df_Fe56.csv')
print(len(df))

Number of rows dropped: 21
4495


In [6]:
def RLRT_extract_qvbin(df=None,bin_index=0,bc=True):
    Ex_increments = np.array([0.0012,0.001,0.0006,0.0006,0.0006,0.0010,0.0036,0.0036,0.0036,0.0036,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002])
    W2_increments = np.array([0.01,0.002,0.002,0.014,0.018,0.01,0.008,0.032,0.06,0.018,0.04,0.12,0.12,0.12,0.12,0.24,0.24,0.24])
    qvcenter = qvcenters[bin_index]
    bin_data = df.loc[df['qvcenter'] == qvcenter].copy()
    fit = pd.DataFrame()
    Ex_inc = Ex_increments[bin_index]
    W2_inc = W2_increments[bin_index]
    Ex = 0.0
    nu = np.sqrt(mass_nucleus**2 + qvcenter**2 + 2 * mass_nucleus * Ex) - mass_nucleus
    nu_upper = np.sqrt(mass_nucleus**2 + qvcenter**2 + 2 * mass_nucleus * (Ex + Ex_inc)) - mass_nucleus
    W2 = mass_nucleon**2 + 2 * mass_nucleon * 0.05 - (qvcenter**2 - 0.05**2)

    while nu <= qvcenter:

        if nu_upper < 0.05:
            picked = bin_data.loc[(bin_data['Ex'] >= Ex) & (bin_data['Ex'] < Ex + Ex_inc)].copy()
            picked['Hbc_Sig(GeV)'] = picked['bc_qv_ex'] * picked['Hcc_Sig(GeV)']
            picked['Hbc_error(GeV)'] = picked['bc_qv_ex'] * picked['Hcc_error(GeV)']
            nu = np.sqrt(mass_nucleus**2 + qvcenter**2 + 2 * mass_nucleus * Ex) - mass_nucleus
            nu_upper = np.sqrt(mass_nucleus**2 + qvcenter**2 + 2 * mass_nucleus * (Ex + Ex_inc)) - mass_nucleus
            Ex += Ex_inc
        else:
            picked = bin_data.loc[(bin_data['W2'] >= W2) & (bin_data['W2'] < W2 + W2_inc)].copy()
            picked['Hbc_Sig(GeV)'] = picked['bc_qv_w2'] * picked['Hcc_Sig(GeV)']
            picked['Hbc_error(GeV)'] = picked['bc_qv_w2'] * picked['Hcc_error(GeV)']
            nu = np.sqrt(qvcenter**2 + W2) - mass_nucleon
            nu_upper = np.sqrt(qvcenter**2 + W2 + W2_inc) - mass_nucleon
            W2 += W2_inc

        nu_mid = (nu + nu_upper) / 2
        q2 = qvcenter**2 - nu_mid**2

        x = np.array(picked["epsilon"].values)
        if bc:
            y = np.array(picked["Hbc_Sig(GeV)"].values)
            y_err = np.array(picked["Hbc_error(GeV)"].values)
        else:
            y = np.array(picked["Hcc_Sig(GeV)"].values)
            y_err = np.array(picked["Hcc_error(GeV)"].values)
       
        if len(y) >= 2 and (np.max(x) - np.min(x)) >= 0.25:
        
            if len(y) == 2:
                x = np.concatenate([x, x])
                y = np.concatenate([y + 0.1 * y_err, y - 0.1 * y_err])
                y_err = y_err * 1.414
                y_err = np.concatenate([y_err, y_err])
                
            params, covariance = curve_fit(linear_model, x, y, sigma=y_err, absolute_sigma=True)

            a_opt, b_opt = params
            a_err, b_err = np.sqrt(np.diag(covariance))
            Chi2 = np.sum(np.square((y - linear_model(x, a_opt, b_opt)) / y_err))
            RL = a_opt / 1000
            RLerr = a_err / 1000 
            RT = (2 * b_opt * q2 / (qvcenter**2)) / 1000
            RTerr = (2 * b_err * q2 / (qvcenter**2)) / 1000
            new_row = pd.Series({'qvcenter':qvcenter,'Q2center':None,'nu':nu_mid,'RL':RL,'RLerr':RLerr,'RT':RT,'RTerr':RTerr,'Chi2':Chi2, 'num_points':len(y),'Chi2/DOF':Chi2/(len(y)-2)})             
            fit = append_row(fit,new_row)
    
    fit['Ex'] = fit['nu'] - (qvcenter**2 - fit['nu']**2) / (2 * mass_nucleus)
    fit['W2'] = mass_nucleon**2 + 2 * mass_nucleon * fit["nu"] - (qvcenter**2 - fit['nu']**2)
    fit['RL'] = fit['RL']*1e3
    fit['RLerr'] = fit['RLerr']*1e3
    fit['RT'] = fit['RT']*1e3
    fit['RTerr'] = fit['RTerr']*1e3
    return fit

def RLRT_extract_Q2bin(df=None,bin_index=0,bc=True):
    Ex_increments=np.array([0.006,0.0025,0.005,0.005,0.003,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005])
    W2_increments=np.array([0.01,0.01,0.01,0.008,0.01,0.008,0.008,0.03,0.015,0.05,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
    
    Q2center = Q2centers[bin_index]
    
    bin_data   = df.loc[df['Q2center'] == Q2center].copy()
    fit = pd.DataFrame()
    Ex_inc = Ex_increments[bin_index]
    W2_inc = W2_increments[bin_index]
    Ex = 0.0
    nu = Ex + Q2center / (2 * mass_nucleus)
    nu_upper = Ex + Ex_inc + Q2center / (2 * mass_nucleus)
    W2 = mass_nucleon**2 + 2 * mass_nucleon * 0.05 - Q2center

    while nu < 3.5:

        if nu_upper < 0.05:
            picked = bin_data.loc[(bin_data['Ex'] >= Ex) & (bin_data['Ex'] < Ex + Ex_inc)].copy()
            picked['Hbc_Sig(GeV)'] = picked['bc_q2_ex'] * picked['Hcc_Sig(GeV)']
            picked['Hbc_error(GeV)'] = picked['bc_q2_ex'] * picked['Hcc_error(GeV)']
            nu = Ex + Q2center / (2 * mass_nucleus)
            nu_upper = Ex + Ex_inc + Q2center / (2 * mass_nucleus) 
            Ex += Ex_inc
        else:
            picked = bin_data.loc[(bin_data['W2'] >= W2) & (bin_data['W2'] < W2 + W2_inc)].copy()
            picked['Hbc_Sig(GeV)'] = picked['bc_q2_w2'] * picked['Hcc_Sig(GeV)']
            picked['Hbc_error(GeV)'] = picked['bc_q2_w2'] * picked['Hcc_error(GeV)']
            nu = (W2 - mass_nucleon**2 + Q2center) / (2 * mass_nucleon)
            nu_upper = (W2 + W2_inc - mass_nucleon**2 + Q2center) / (2 * mass_nucleon)
            W2 += W2_inc

        nu_mid = (nu + nu_upper) / 2
        qv = np.sqrt(Q2center + ((nu_upper + nu) / 2)**2)
        x = np.array(picked["epsilon"].values)
        if bc:
            y = np.array(picked["Hbc_Sig(GeV)"].values)
            y_err = np.array(picked["Hbc_error(GeV)"].values)
        else:
            y = np.array(picked["Hcc_Sig(GeV)"].values)
            y_err = np.array(picked["Hcc_error(GeV)"].values)
        
        if len(y) >= 2 and (np.max(x) - np.min(x)) >= 0.25:
            if len(y) == 2:
                x = np.concatenate([x, x])
                y = np.concatenate([y + 0.1 * y_err, y - 0.1 * y_err])
                y_err = y_err * 1.414
                y_err = np.concatenate([y_err, y_err])
                
            params, covariance = curve_fit(linear_model, x, y, sigma=y_err, absolute_sigma=True)
            a_opt, b_opt = params
            a_err, b_err = np.sqrt(np.diag(covariance))
            Chi2 = np.sum(np.square((y-linear_model(x,a_opt,b_opt))/y_err))
            RL = a_opt/1000
            RLerr = a_err/1000 
            RT = (2*b_opt*Q2center/(qv**2))/1000
            RTerr = (2*b_err*Q2center/(qv**2))/1000
            new_row = pd.Series({'qvcenter':None,'Q2center':Q2center,'nu':nu_mid,'RL':RL,'RLerr':RLerr,'RT':RT,'RTerr':RTerr,'Chi2':Chi2,'num_points':len(y),'Chi2/DOF':Chi2/(len(y)-2)})
            fit = append_row(fit,new_row)

    fit['Ex'] = fit['nu'] - Q2center / (2 * mass_nucleus)
    fit['W2'] = mass_nucleon**2 + 2 * mass_nucleon * fit["nu"] - Q2center
    fit['RL'] = fit['RL']*1e3
    fit['RLerr'] = fit['RLerr']*1e3
    fit['RT'] = fit['RT']*1e3
    fit['RTerr'] = fit['RTerr']*1e3
    return fit


In [7]:
df_Fe56_thisAnalysis = pd.DataFrame()
qv_bin_indices=[3,4,5,6,7,8,9,10,11,12,13,14,15,16]
Q2_bin_indices=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
for i in range(len(qv_bin_indices)):
        bin_index = qv_bin_indices[i]
        fit = RLRT_extract_qvbin(df=df,bin_index=bin_index)
        df_Fe56_thisAnalysis = pd.concat([df_Fe56_thisAnalysis, fit], ignore_index=True)
for i in range(len(Q2_bin_indices)):
        bin_index = Q2_bin_indices[i]
        fit = RLRT_extract_Q2bin(df=df,bin_index=bin_index)
        df_Fe56_thisAnalysis = pd.concat([df_Fe56_thisAnalysis, fit], ignore_index=True)
df_Fe56_thisAnalysis.to_csv('Output/df_Fe56_thisAnalysis.csv',index=False)